# World Cup 2022 winner classifier
Imagine being able to predict which team will win during a match. All your bets would win and you'd be rich! Unfortunately, football, like any sport, is unpredictable, especially at the World Cup.


An attempt to build such an ML model is doomed to failure, the only question is how much? The project's task is to predict the winner of a given match, and thanks to the ongoing championships, I can check the accuracy of the model.

Datasets used: 
FIFA ranking -  https://www.kaggle.com/code/agostontorok/soccer-world-cup-2018-winner/data
football results -  https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [137]:
#importing data
fifa_ranking = pd.read_csv(r"data\world cup 2022\fifa_ranking.csv")
matches =  pd.read_csv(r"data\world cup 2022\results.csv")

### matches data 

In [138]:
matches

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44201,2022-12-01,Canada,Morocco,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44202,2022-12-02,Serbia,Switzerland,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44203,2022-12-02,Cameroon,Brazil,NaN,NaN,FIFA World Cup,Lusail,Qatar,True
44204,2022-12-02,Ghana,Uruguay,NaN,NaN,FIFA World Cup,Al Wakrah,Qatar,True


In [139]:
#renaming columns 
matches.rename(columns = {'home_team':'team_1', 'away_team':'team_2','home_score':'team_1_score', 'away_score':'team_2_score'}, inplace = True)

In [140]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44206 entries, 0 to 44205
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          44206 non-null  object 
 1   team_1        44206 non-null  object 
 2   team_2        44206 non-null  object 
 3   team_1_score  44186 non-null  float64
 4   team_2_score  44186 non-null  float64
 5   tournament    44206 non-null  object 
 6   city          44206 non-null  object 
 7   country       44206 non-null  object 
 8   neutral       44206 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


In [141]:
# date dtype is object, let's convert it to datetime
matches["date"] = pd.to_datetime(matches["date"], format = '%Y-%m-%d')

In [142]:
# these matches have not yet taken place or have taken place but on current world cup, they need to be deleted
matches.tail(20)

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral
44186,2022-11-28,Cameroon,Serbia,NaN,NaN,FIFA World Cup,Al Wakrah,Qatar,True
44187,2022-11-28,Brazil,Switzerland,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44188,2022-11-28,South Korea,Ghana,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True
44189,2022-11-28,Portugal,Uruguay,NaN,NaN,FIFA World Cup,Lusail,Qatar,True
44190,2022-11-29,Ecuador,Senegal,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True
44191,2022-11-29,Qatar,Netherlands,NaN,NaN,FIFA World Cup,Al Khor,Qatar,False
44192,2022-11-29,Wales,England,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True
44193,2022-11-29,Iran,United States,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44194,2022-11-30,Poland,Argentina,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44195,2022-11-30,Saudi Arabia,Mexico,NaN,NaN,FIFA World Cup,Lusail,Qatar,True


In [143]:
matches.drop(matches.index[44158:], axis=0, inplace=True )
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44153,2022-11-20,Slovenia,Montenegro,1.0,0.0,Friendly,Ljubljana,Slovenia,False
44154,2022-11-20,South Africa,Angola,1.0,1.0,Friendly,Mbombela,South Africa,False
44155,2022-11-20,Saint Lucia,San Marino,1.0,0.0,Friendly,Gros Islet,Saint Lucia,False
44156,2022-11-20,Syria,Venezuela,1.0,2.0,Friendly,Dubai,United Arab Emirates,True


### ranking data

In [144]:
fifa_ranking

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57788,206,Anguilla,AIA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57789,206,Bahamas,BAH,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57790,206,Eritrea,ERI,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07


In [145]:
#renaming columns 
fifa_ranking.rename(columns = {'rank_date':'date'}, inplace = True)

In [146]:
fifa_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   rank                     57793 non-null  int64  
 1   country_full             57793 non-null  object 
 2   country_abrv             57793 non-null  object 
 3   total_points             57793 non-null  float64
 4   previous_points          57793 non-null  int64  
 5   rank_change              57793 non-null  int64  
 6   cur_year_avg             57793 non-null  float64
 7   cur_year_avg_weighted    57793 non-null  float64
 8   last_year_avg            57793 non-null  float64
 9   last_year_avg_weighted   57793 non-null  float64
 10  two_year_ago_avg         57793 non-null  float64
 11  two_year_ago_weighted    57793 non-null  float64
 12  three_year_ago_avg       57793 non-null  float64
 13  three_year_ago_weighted  57793 non-null  float64
 14  confederation         

In [147]:
# date dtype is object, let's convert it to datetime
fifa_ranking["date"] = pd.to_datetime(fifa_ranking["date"], format = '%Y-%m-%d')

### final data set

In [149]:
#making target variable
matches["result"] = 1
matches.loc[matches.team_1_score > matches.team_2_score, "result" ] = 0
matches.loc[matches.team_1_score < matches.team_2_score, "result" ] = 2

In [150]:
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,1
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,0
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,0
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,1
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,0
...,...,...,...,...,...,...,...,...,...,...
44153,2022-11-20,Slovenia,Montenegro,1.0,0.0,Friendly,Ljubljana,Slovenia,False,0
44154,2022-11-20,South Africa,Angola,1.0,1.0,Friendly,Mbombela,South Africa,False,1
44155,2022-11-20,Saint Lucia,San Marino,1.0,0.0,Friendly,Gros Islet,Saint Lucia,False,0
44156,2022-11-20,Syria,Venezuela,1.0,2.0,Friendly,Dubai,United Arab Emirates,True,2


In [151]:
#the world cup is on a neutral ground, so we should remove the other options
# However, it is worth keeping the data. We will see later if this was good choice.

In [152]:
#the first ranking date appeared in 1993, because of that we have to drop matches before this date
fifa_ranking.sort_values("date")

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,date
0,1,Germany,GER,0.00,57,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,UEFA,1993-08-08
107,108,Lebanon,LIB,0.00,0,53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,AFC,1993-08-08
108,109,South Africa,RSA,0.00,4,15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,1993-08-08
109,110,Luxembourg,LUX,0.00,9,-7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,UEFA,1993-08-08
110,111,Faroe Islands,FRO,0.00,11,-17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,UEFA,1993-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57655,74,South Africa,RSA,427.68,432,-2,186.63,186.63,204.55,102.28,268.66,80.60,290.93,58.19,CAF,2018-06-07
57656,75,China PR,CHN,423.09,431,-2,204.03,204.03,193.60,96.80,243.84,73.15,245.50,49.10,AFC,2018-06-07
57657,76,Zambia,ZAM,420.31,412,2,236.14,236.14,137.37,68.68,186.75,56.02,297.31,59.46,CAF,2018-06-07
57659,78,Belarus,BLR,417.42,410,1,204.80,204.80,169.84,84.92,294.64,88.39,196.50,39.30,UEFA,2018-06-07


In [153]:
first_date = fifa_ranking.iloc[0].date
matches = matches[matches.date >= first_date]

In [154]:
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result
18256,1993-08-08,Bolivia,Uruguay,3.0,1.0,FIFA World Cup qualification,La Paz,Bolivia,False,0
18257,1993-08-08,Brazil,Mexico,1.0,1.0,Friendly,Maceió,Brazil,False,1
18258,1993-08-08,Ecuador,Venezuela,5.0,0.0,FIFA World Cup qualification,Quito,Ecuador,False,0
18259,1993-08-08,Guinea,Sierra Leone,1.0,0.0,Friendly,Conakry,Guinea,False,0
18260,1993-08-08,Paraguay,Argentina,1.0,3.0,FIFA World Cup qualification,Asunción,Paraguay,False,2
...,...,...,...,...,...,...,...,...,...,...
44153,2022-11-20,Slovenia,Montenegro,1.0,0.0,Friendly,Ljubljana,Slovenia,False,0
44154,2022-11-20,South Africa,Angola,1.0,1.0,Friendly,Mbombela,South Africa,False,1
44155,2022-11-20,Saint Lucia,San Marino,1.0,0.0,Friendly,Gros Islet,Saint Lucia,False,0
44156,2022-11-20,Syria,Venezuela,1.0,2.0,Friendly,Dubai,United Arab Emirates,True,2


In [163]:
#writing a function to assign the appropriate index from the fifa_ranking df, where the team's rank is located
def assign_rank(team1, team2, date):
    ranking = fifa_ranking[fifa_ranking.date <= date]
    rank_team1 = ranking[ranking.country_full == team1]
    rank_team2 = ranking[ranking.country_full == team2]
    
    if(rank_team1.empty):
        team1_index = np.NaN
    else:
        team1_index = rank_team1.sort_values("date", ascending=False).index[0]
        
    if(rank_team2.empty):
        team2_index = np.NaN
    else:
        team2_index = rank_team2.sort_values("date", ascending=False).index[0]
        
    return pd.Series([team1_index, team2_index])

In [161]:
#it seems to be working
assign_rank(matches.iloc[-1].team_1, matches.iloc[-1].team_2, matches.iloc[-1].date)

0    57676
1    57651
dtype: int32

In [174]:
#scale it up 
matches[["id1", "id2"]] = matches.apply(lambda row: assign_rank(row["team_1"], row["team_2"], row["date"]), axis=1)

D:\Anaconda_pliki_instalacyjne\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [175]:
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result,id1,id2
18256,1993-08-08,Bolivia,Uruguay,3.0,1.0,FIFA World Cup qualification,La Paz,Bolivia,False,0,58.0,21.0
18257,1993-08-08,Brazil,Mexico,1.0,1.0,Friendly,Maceió,Brazil,False,1,7.0,13.0
18258,1993-08-08,Ecuador,Venezuela,5.0,0.0,FIFA World Cup qualification,Quito,Ecuador,False,0,34.0,93.0
18259,1993-08-08,Guinea,Sierra Leone,1.0,0.0,Friendly,Conakry,Guinea,False,0,64.0,85.0
18260,1993-08-08,Paraguay,Argentina,1.0,3.0,FIFA World Cup qualification,Asunción,Paraguay,False,2,66.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
44153,2022-11-20,Slovenia,Montenegro,1.0,0.0,Friendly,Ljubljana,Slovenia,False,0,57637.0,57624.0
44154,2022-11-20,South Africa,Angola,1.0,1.0,Friendly,Mbombela,South Africa,False,1,57655.0,57718.0
44155,2022-11-20,Saint Lucia,San Marino,1.0,0.0,Friendly,Gros Islet,Saint Lucia,False,0,NaN,57784.0
44156,2022-11-20,Syria,Venezuela,1.0,2.0,Friendly,Dubai,United Arab Emirates,True,2,57654.0,57614.0


In [182]:
#some of the values are NaN, let's see why
matches.loc[matches.isnull().values, ["date", "team_1", "team_2", "id1", "id2" ]]
#looks like first nan values are due to the lack of country in ranking (future rankings include more countries)

,date,team_1,team_2,id1,id2
18262,1993-08-08,Zimbabwe,Eswatini,49.0,NaN
18282,1993-08-31,Iceland,United States,52.0,NaN
18288,1993-09-06,Eswatini,Zimbabwe,NaN,49.0
18298,1993-09-08,Norway,United States,8.0,NaN
18301,1993-09-08,Wales,Czech Republic,26.0,NaN
...,...,...,...,...,...
44113,2022-11-17,North Macedonia,Finland,NaN,57644.0
44119,2022-11-17,Saint Lucia,San Marino,NaN,57784.0
44131,2022-11-19,Ivory Coast,Burkina Faso,NaN,57633.0
44149,2022-11-20,North Macedonia,Azerbaijan,NaN,57686.0


In [209]:
#but what about last nan values? Maybe problem is with country name?
# matches.loc[matches.id1.isnull().values, "team_1"].value_counts()[30:40]
matches.loc[matches.id2.isnull().values, "team_2"].value_counts()[40:]

Bonaire                     15
Northern Mariana Islands    14
Székely Land                14
Iraqi Kurdistan             14
Sint Maarten                13
                            ..
Andalusia                    1
Brittany                     1
Samoa                        1
Dominica                     1
Aymara                       1
Name: team_2, Length: 95, dtype: int64

In [200]:
np.sort(fifa_ranking.country_full.unique())

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada',
       'Cape Verde Islands', 'Cayman Islands', 'Central African Republic',
       'Chad', 'Chile', 'China PR', 'Chinese Taipei', 'Colombia',
       'Comoros', 'Congo', 'Congo DR', 'Cook Islands', 'Costa Rica',
       'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic',
       "Côte d'Ivoire", 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'England',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia',
  

In [210]:
#United States is USA in FIFA
#South Korea is Korea Republic
#Iran is IR Iran
#Ivory Coast is Côte d'Ivoire
#North Macedonia is  FYR Macedonia
#DR Congo  is Congo or Congo DR
#North Korea is Korea DPR
#Northern Cyprus is Cyprus
#Eswatini is Swaziland
#rest are insignificant teams and I will drop them